# Web Scraping with Selenium
*Author: Douglas Strodtman (SaMo)*

## Learning Objectives

1. Scraping Javascript pages
2. Manipulating objects on a page
3. Automating logins
4. Passing cookies to `requests`

## Installs

#### Required Software

- Google Chrome
- [Xpath Helper](https://chrome.google.com/webstore/detail/xpath-helper/hgimnogjllphhhkhlmebbmlgjoejdpjl?hl=en)
- [Chromedriver](http://chromedriver.chromium.org/downloads) (Download the Chromedriver for your OS. Unzip and move the `chromedriver` file to the directory containing this notebook.)

#### Python 
- scrapy
- selenium
- beautiful soup

Uncomment and run the install for any of the packages you're missing below.

In [85]:
# !pip install scrapy
# !pip install seleniumnn 
# !pip install beautifulsoup4 

In [86]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from scrapy.selector import Selector
import re
import time
import csv
from itertools import islice

## Using Selenium to Automate Logins

To avoid typing my password in the browser, I created a hidden file with the variable

```PASSWORD = 'myPassw0rd'``` 

defined. I add this file to my `.gitignore` so it doesn't get accidentally shared when I push to Github. **I'm not saying this is security best practices, but it's a simple safeguard.**

In [87]:
%run .password.py

ERROR:root:File `'.password.py'` not found.


### Headless Browsing
One of the great options available with Selenium is the ability to automate websurfing without open a visual browser. This is as simple as adding

```options = webdriver.ChromeOptions()
options.add_argument('headless')```

### Logging in to GHE
All uses of Selenium are esoteric. If you want to log in to a site, build out a custom function that utilizes that site's structure and functionality and return the logged in driver.

In [88]:
def login_to_github(username, user_pass, headless=False, repo='DSI-US-5/course-info'):
    if headless:
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        driver = webdriver.Chrome('./chromedriver', chrome_options=options)
    else:
        driver = webdriver.Chrome('./chromedriver')
    driver.get(f'https://git.generalassemb.ly/{repo}')
    
    

    user = driver.find_elements_by_css_selector('input[type=text]')[0]
    user.send_keys(username)

    password = driver.find_element_by_css_selector('input[type=password]')
    password.send_keys(user_pass)

    button = driver.find_element_by_css_selector('.btn')
    button.click()
    
    

    return driver

In [89]:
driver = login_to_github('dstrodtman', PASSWORD)

NameError: name 'PASSWORD' is not defined

Notice that this opens a new Chrome browser window. I can use xpath (or other options) to select elements of this page.

In [ ]:
insights = driver.find_element_by_xpath("//a[@class='js-selected-navigation-item reponav-item'][3]")

In [ ]:
insights.text

I can now automate a click to navigate.

In [ ]:
insights.click()

Notice that I've now navigated to a different page in my browser.

In [ ]:
driver.quit()

In [ ]:
del driver

### Passing Cookies to Requests
While Selenium is powerful, it can be much slower than `requests`. Whenever possible, accelerate your scraping by capturing the html source instead of rendering the Javascript. You can pass cookies that are generated automatically back to `requests` to mimic browser behavior.

In [110]:
driver = webdriver.Chrome('./chromedriver')

In [111]:
driver.get(f'https://www.6pm.com/marty/c/homepage-new')

In [ ]:
engine = sqlalchemy.create_engine('192.168.0001:5000/$password|postgres@ubuntu.local')
conn = engine.connect()

In [ ]:
conn.execute('SELECT * FROM mytable;')
responses = conn.fetchall()

outputs = [line for line in responses]

In [112]:
def get_cookie_jar(driver):
    cookies = driver.get_cookies()
    cookie_jar = {x['name']:x['value'] for x in cookies}
    
    return cookie_jar

In [113]:
cookie_jar = get_cookie_jar(driver)

In [114]:
cookie_jar

{'_gid': 'GA1.2.371348830.1537995573',
 'zfc': 'ChYI8oW4veEsEPL28sipigIY8oW4veEsEgwKBG1wbHMQARgHIAESDAoEc2F1aRABGAUgARILCgNtc2MQARgFIAE=',
 'cloud': 'west2',
 '_ga': 'GA1.2.1902316271.1537995573',
 'ak_bmsc': '3B5C880E50C32693AA9B95A12BA1C95E3F97771ECA3D000034F3AB5B2CB9F635~pl4roICzHiyzzdJ/3Or1o9VxYufR+6+nspt8FmdnupmSDP+VMmTq29lxYqXRAnzDnIN5DEpqbO0V1mt3TU2xltV3dQuuu59rUNjvFLD6W/DTEvraEnsRhPiYGJOsPyn2qhP4YEn8hIcUzXd0R3dWV/5zKW8wvCvMmG1yFaPcYXBWrFux77HHz6unQfDfVL7v3O9b50h6XIO6xo/DEEbzfCEBHEijYh7wsXgYO5kkwCWkE=',
 'deviceType': 'desktop',
 '_gat_6pm': '1',
 'AKA_A2': 'A',
 'geo': 'US/CA/807/San%20Francisco',
 'tid': '4bERnJ4ugzgPddqilC6O9FXeExLNyBD8',
 'clouddc': 'west2'}

I pass these cookies back to requests using the `cookies` arg.

In [115]:
page = requests.get('https://www.6pm.com/marty/c/homepage-new', 
                    cookies=cookie_jar)

In [116]:
soup = BeautifulSoup(page.text, 'html.parser')

In [117]:
driver.current_url

'https://www.6pm.com/marty/c/homepage-new'

In [118]:
driver.find_element_by_xpath("/html/body[@class='activeMain welg nav-close']/div[@id='root']//main//div[@id='productRecap']/div[@id='stage']//button/img/@src")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body[@class='activeMain welg nav-close']/div[@id='root']//main//div[@id='productRecap']/div[@id='stage']//button/img/@src"}
  (Session info: chrome=69.0.3497.100)
  (Driver info: chromedriver=2.42.591059 (a3d9684d10d61aa0c45f6723b327283be1ebaad8),platform=Mac OS X 10.13.6 x86_64)


In [120]:
driver.find_element_by_xpath("/html/body[@class='activeMain welg nav-close']/div[@id='root']//main//button/img/@src")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body[@class='activeMain welg nav-close']/div[@id='root']//main//button/img/@src"}
  (Session info: chrome=69.0.3497.100)
  (Driver info: chromedriver=2.42.591059 (a3d9684d10d61aa0c45f6723b327283be1ebaad8),platform=Mac OS X 10.13.6 x86_64)


In [124]:
driver.find_element_by_xpath("/html/body//main//img/@src")

InvalidSelectorException: Message: invalid selector: The result of the xpath expression "/html/body//main//img/@src" is: [object Attr]. It should be an element.
  (Session info: chrome=69.0.3497.100)
  (Driver info: chromedriver=2.42.591059 (a3d9684d10d61aa0c45f6723b327283be1ebaad8),platform=Mac OS X 10.13.6 x86_64)


In [102]:
driver.find_element_by_xpath("/html/body[@class='activeMain welg nav-close']/div[@id='root']/div/div[@class='_17sMd']/main[@class='_3qw1r mdl-layout__content']/div/div/div[@class='_2TnGa pdpTestWrapper']/div[@class='_38is3']/div[@class='_3yLTA']/div[@id='productRecap']/div[@id='stage']/div[@class='_2wOyj']/div/div[@class='DyCfm']/div[@class='_2Sa9O']/div[@class='tIIJH']/div[@class='_3L_Gh']/div[@class='_ftgE']/button[@class='_2PlG6']/img[@class='_1C2ks']/@src")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body[@class='activeMain welg nav-close']/div[@id='root']/div/div[@class='_17sMd']/main[@class='_3qw1r mdl-layout__content']/div/div/div[@class='_2TnGa pdpTestWrapper']/div[@class='_38is3']/div[@class='_3yLTA']/div[@id='productRecap']/div[@id='stage']/div[@class='_2wOyj']/div/div[@class='DyCfm']/div[@class='_2Sa9O']/div[@class='tIIJH']/div[@class='_3L_Gh']/div[@class='_ftgE']/button[@class='_2PlG6']/img[@class='_1C2ks']/@src"}
  (Session info: chrome=69.0.3497.100)
  (Driver info: chromedriver=2.42.591059 (a3d9684d10d61aa0c45f6723b327283be1ebaad8),platform=Mac OS X 10.13.6 x86_64)


In [ ]:
'''
<a class="gae-click*Main-Nav*Shoes*Women-s-Boots" href="/women-boots/CK_XARCz1wHAAQHiAgMYAQI.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/">Boots</a>
'''

In [ ]:
soup.findAll('a',{'class':'gae-click*Main-Nav*Shoes*Women-s-Boots'})

In [ ]:
URL = "http://6pm.com/women-boots/CK_XARCz1wHAAQHiAgMYAQI.zso?s=isNew/desc/goLiveDate/desc/recentSalesStyle/desc/"
page1 = requests.get(URL,
                     cookies = cookie_jar)
page1

In [ ]:
soup1.findALL('a', {})

In [ ]:
user_shoe_size = 5.5
shoe_size = driver.find_elements_by_css_selector('input[type=text]')[0]
shoe_size.send_keys(user_shoe_size)

In [ ]:
links = []
for link in soup1.find_all('a', {'itemprop':'url'}):
    links.append(link.get('href'))
    

In [ ]:
soup.find_all('a',{'rel':'next'})[0]['href']

In [ ]:
soup1.find_all('a',{'rel':'next'})[0]['href']

In [ ]:
links

In this simple example, I'll just extract the total number of commits to this page (here, I use `Selector` from `scrapy` to select by xpath so that my process for using `requests` is more similar to that for `selenium`.

In [ ]:
elector(text=soup1.text).xpath("/html/body[@class='activeMain welg nav-close']/div[@id='root']/div/div[@class='_17sMd']/main[@class='_3qw1r mdl-layout__content']/div/div/div[@class='_2TnGa pdpTestWrapper']/div[@class='_38is3']/div[@class='_3yLTA']/div[@id='productRecap']/div[@id='stage']/div[@class='_2wOyj']/div/div[@class='DyCfm']/div[@class='_2Sa9O']/div[@class='tIIJH']/div[@class='_3L_Gh']/div[@class='_ftgE']/button[@class='_2PlG6']/img[@class='_1C2ks']/@src").extract()[0]

In [40]:
commits = Selector(text=page.text).xpath("//li[@class='commits']/a/span").extract()[0]

IndexError: list index out of range

In [17]:
commits

NameError: name 'commits' is not defined

As you can see, my result isn't as clean as I might like, but with some quick `BeautifulSoup` and regex parsing, I can clean things up. **In general**, my preferenece is to get as much data as I can in a single call and then clean it using the same reusable functions.

In [18]:
re.sub('\s*', '', BeautifulSoup(commits, 'html.parser').text)

NameError: name 'commits' is not defined

## Using Selenium to Load Javascript Pages

The below functions are esoteric to boardgamegeek.com, which is the website that I chose to scrape for my capstone project. My approach was to maximize the amount of navigation I could do with `requests` and only use `selenium` for loading dynamically generated pages so that I could capture the data from their page source. There are **many** other ways to approach these problems, and I'm not suggesting that my solution is the best, just that it is a workable solution.

#### This function defines my driver for a single page
I choose to open a new driver for each page that I visit, rather than automating button clicks.

In [19]:
def connect_to_bgg(glink, curr_page, headless=True):
    if headless:
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        driver = webdriver.Chrome('./chromedriver', chrome_options=options)
    else:
        driver = webdriver.Chrome('./chromedriver')
    driver.get(f'https://boardgamegeek.com{glink}/ratings?pageid={curr_page}&rated=1')

    return driver

#### This function allows me to check that my page has finished loading
I was running into issues where I was trying to scrape before the page had finished loading, which at various times in my troubleshooting led to either just errors or (in an earlier iteration of this approach) scraping the results from the previous page that I had visited. While this function is all error handling, I'm programming around expected error behavior based upon what I know about the page.

In [20]:
def check_page_loaded(driver, last_page_el):
    first_rater = []
    page_el = []
    while not first_rater or not page_el:
        try:
            first_rater = driver.find_element_by_xpath("//ratings-module//li[@class='summary-item summary-rating-item']\
                                                        [1]/div[@class='comment-header']/div/div/a")
            page_el = first_rater.text
        except:
            time.sleep(.5)
    while page_el == last_page_el:
        try:
            first_rater = driver.find_element_by_xpath("//ratings-module//li[@class='summary-item summary-rating-item']\
                                                        [1]/div[@class='comment-header']/div/div/a")
            page_el = first_rater.text
        except:
            time.sleep(.5)
            
    return page_el

#### Setting up the lists of items that I need to collect
I had previously scraped the top 10k boardgames using `requests` from ranked lists that returned simple html. I was able to capture a unique identifier number(`gid`), the specific url path to that game (`glink`), and the total number of users that had provided a rating for that game (`numrating`).

In [21]:
def read_gid_link_numratings(start=0, stop=10000):
    gids = []
    glinks = []
    numratings = []
    with open('data/numratings', 'r') as f:
        reader = csv.reader(islice(f, start, stop+1))
        for row in reader:
            gids.append(row[0])
            glinks.append(row[1])
            numratings.append(row[2])
    return gids, glinks, numratings

#### Define max pages
To avoid having to automate clicks, I chose to leverage the design of the site to iterate through pages. Here, I find the number of pages of reviews that I should expect for each game.

In [22]:
def set_max_pages(numrating):
    numrating = int(numrating)
    if numrating%50 != 0:
        max_pages = numrating//50 + 1
    else:
        max_pages = int(numrating/50)
    return max_pages

#### Create user tuples
Because I knew that I wanted my data to end up in a normalized postgres database, I chose to include all necessary info here to define my table. Both `user_name` and `gid` are unique identifiers, so I use these to index ratings.

In [23]:
def make_user_rows(user_names, gid, ratings):
    return list(zip(user_names, [gid]*len(user_names), ratings))

#### Write tuples to CSV
I write out my data after **every** query into a file for each game. Reasons:
- For my scrape, I knew that I needed to wait 2s between calls so as to not overload the site
- This avoids the possibility of filling up my RAM and crashing my instance
- I cannot lose any data to unknown errors (at most, I only lose a single page query worth of data)
- My files will be of expected len (my max `numratings` for a game is only around 70k, so my files won't get huge)

In [24]:
def write_user_rows(gid,user_rows):
    with open(f'data/{gid}_users', 'a+') as f:
        csv.writer(f).writerows(user_rows)

#### Bring it all together
Here I build out a function to combine these smaller functions to grab all the ratings for a single game. Note that I have both a log file and I print out my progress along the way. This helps me in knowing when my attempts fail (as they certainly will).

In [25]:
def get_game_raters(gid, glink, max_pages, curr_page=1):

    last_page_el = []

    while curr_page <= max_pages:
        start = time.time()

        driver = connect_to_bgg(glink, curr_page)
        
        last_page_el = check_page_loaded(driver, last_page_el)
        
        html = driver.page_source
        
        driver.quit()

        xpath_user_names = "//ratings-module//div[@class='comment-header']/div/div/a/text()"
        xpath_user_links = "//ratings-module//div[@class='comment-header']/div/div/a/@href"
        xpath_ratings = "//ratings-module//li/div[@class='summary-item-callout']/div/text()"

        user_names = Selector(text=html).xpath(xpath_user_names).extract()
        user_links = Selector(text=html).xpath(xpath_user_links).extract()
        dirty_ratings = Selector(text=html).xpath(xpath_ratings).extract()

        ratings = []
        for rating in dirty_ratings:
            ratings.append(re.sub('\s', '', rating))

        user_rows = make_user_rows(user_names, gid, ratings)

        write_user_rows(gid,user_rows)
        
        print(f'Scraped {gid} page {curr_page} of {max_pages} in {time.time()-start}s')
        
        curr_page += 1

    with open('get_users_log', 'a+') as f:
        f.write(f'{time.time()} {gid} finished\n')    

#### Here, we'll grab all the ratings for a single game
My `gids` are ordered by `numratings`, so I choose a game a little further in so this doesn't take forever.

In [26]:
gids, glinks, numratings = read_gid_link_numratings()

glink = glinks[8000]
gid = gids[8000]
max_pages = set_max_pages(numratings[8000])

FileNotFoundError: [Errno 2] No such file or directory: 'data/numratings'

In [27]:
get_game_raters(gid, glink, max_pages)

NameError: name 'gid' is not defined

#### A final wrapper function
Here I build out a final function to wrap all of my inside functions and iterate through a set amount of games.

**NOTE**: It will take roughly two weeks for this scrape to complete. I managed to cut this time to around 4 days by splitting this up onto 10 AWS instances.

**In addition**, some of the paths in my `glinks` have been corrupted due to internal changes in board game names that will result in infinite while loops with my current code.

In [28]:
def get_user_ratings(start=0, stop=10000):
    gids, glinks, numratings = read_gid_link_numratings(start, stop)
    for gid, glink, numrating in zip(gids, glinks, numratings):
        max_pages = set_max_pages(numrating)
        get_game_raters(gid, glink, max_pages)